In [5]:
import pickle
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import types
from scipy.stats import mannwhitneyu
import statsmodels.api as sm
import pylab as pl
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from decimal import *
getcontext().prec = 30
import math

In [6]:
with open("/home/allan/Desktop/enron_fraud_detection/final_project_dataset.pkl","r") as f:
    data_dict = pickle.load(f)
data_dict.pop('TOTAL')
data_dict.pop('THE TRAVEL AGENCY IN THE PARK')

column_names =["name"]+[category for category in data_dict.values()[0].keys()]
enron = pd.DataFrame(columns=column_names)

i=0
for name,values in data_dict.items():
    datum_list={'name':name}
    for key,value in values.items():
        if value=="NaN":
            datum_list[key]=np.nan
        elif not isinstance(value,basestring):
            datum_list[key]=float(value)
        else:
            datum_list[key]=value
    enron=enron.append(pd.DataFrame(data=datum_list,index=[i]))
    i+=1

In [9]:
enron

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,...,name,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,600000,NaN,NaN,NaN,mark.metts@enron.com,NaN,94299,29,38,1,...,METTS MARK,1740,0,585062,NaN,365788,702,807,1061827,585062
1,1200000,1295738,-1386055,NaN,NaN,6680544,11200,NaN,NaN,NaN,...,BAXTER JOHN C,2660303,0,3942714,NaN,267102,NaN,NaN,5634343,10623258
2,350000,NaN,-400729,NaN,steven.elliott@enron.com,4890344,78552,NaN,NaN,NaN,...,ELLIOTT STEVEN,12961,0,1788391,NaN,170941,NaN,NaN,211725,6678735
3,NaN,NaN,NaN,NaN,bill.cordes@enron.com,651850,NaN,12,10,0,...,CORDES WILLIAM R,NaN,0,386335,NaN,NaN,58,764,NaN,1038185
4,1500000,NaN,-3117011,NaN,kevin.hannon@enron.com,5538001,34039,32,32,21,...,HANNON KEVIN P,11350,1,853064,NaN,243293,1035,1045,288682,6391065
5,325000,NaN,NaN,NaN,kristina.mordaunt@enron.com,NaN,35018,NaN,NaN,NaN,...,MORDAUNT KRISTINA M,1411,0,208510,NaN,267093,NaN,NaN,628522,208510
6,NaN,1848227,NaN,NaN,rockford.meyer@enron.com,493489,NaN,28,0,0,...,MEYER ROCKFORD G,NaN,0,462384,NaN,NaN,22,232,1848227,955873
7,2600000,NaN,NaN,NaN,jeffrey.mcmahon@enron.com,1104054,137108,48,58,26,...,MCMAHON JEFFREY,297353,0,558801,NaN,370448,2228,2355,4099771,1662855
8,NaN,3131860,NaN,NaN,stanley.horton@enron.com,5210569,NaN,1073,44,15,...,HORTON STANLEY C,NaN,0,2046079,NaN,NaN,1074,2350,3131860,7256648
9,400000,1130036,-33333,NaN,greg.piper@enron.com,880290,43057,222,61,48,...,PIPER GREGORY F,778,0,409554,-409554,197091,742,1238,1737629,880290


In [36]:
enron_no_outliers=enron.copy()
enron_outliers=enron.copy()

for column in enron.loc[:,(enron.dtypes == np.float64)]:
    outliers = np.abs(enron[column])>np.abs(enron[column]).quantile(0.9)
    mean_no_outliers = enron[~outliers][column].mean()
    enron_no_outliers.loc[outliers,(column)]=mean_no_outliers
    enron_outliers.loc[~outliers,(column)]=0

In [40]:
enron_no_outliers[enron["poi"]==1.]["deferred_income"].plot(kind='hist', alpha=0.5, bins=60)
enron_no_outliers[enron["poi"]==0.]["deferred_income"].plot(kind='hist', alpha=0.5, bins=60)
plt.legend(["poi","non_poi"])
plt.ylabel("Frequency")
plt.title("deferred_income")
plt.show()
plt.figure()
enron_no_outliers[enron["poi"]==1.]["other"].plot(kind='hist', alpha=0.5, bins=60)
enron_no_outliers[enron["poi"]==0.]["other"].plot(kind='hist', alpha=0.5, bins=60)
plt.legend(["poi","non_poi"])
plt.ylabel("Frequency")
plt.title("other")
plt.show()
plt.figure()
enron_no_outliers[enron["poi"]==1.]["expenses"].plot(kind='hist', alpha=0.5, bins=60)
enron_no_outliers[enron["poi"]==0.]["expenses"].plot(kind='hist', alpha=0.5, bins=60)
plt.legend(["poi","non_poi"])
plt.ylabel("Frequency")
plt.title("expenses")
plt.show()
plt.figure()
enron_no_outliers[enron["poi"]==1.]["exercised_stock_options"].plot(kind='hist', alpha=0.5, bins=60)
enron_no_outliers[enron["poi"]==0.]["exercised_stock_options"].plot(kind='hist', alpha=0.5, bins=60)
plt.legend(["poi","non_poi"])
plt.ylabel("Frequency")
plt.title("exercised_stock_options")
plt.show()
plt.figure()
enron_no_outliers[enron["poi"]==1.]["long_term_incentive"].plot(kind='hist', alpha=0.5, bins=60)
enron_no_outliers[enron["poi"]==0.]["long_term_incentive"].plot(kind='hist', alpha=0.5, bins=60)
plt.legend(["poi","non_poi"])
plt.ylabel("Frequency")
plt.title("long_term_incentive")
plt.show()
plt.figure()
enron_no_outliers[enron["poi"]==1.]["bonus"].plot(kind='hist', alpha=0.5, bins=60)
enron_no_outliers[enron["poi"]==0.]["bonus"].plot(kind='hist', alpha=0.5, bins=60)
plt.legend(["poi","non_poi"])
plt.ylabel("Frequency")
plt.title("bonus")
plt.show()
plt.figure()
enron_no_outliers[enron["poi"]==1.]["salary"].plot(kind='hist', alpha=0.5, bins=60)
enron_no_outliers[enron["poi"]==0.]["salary"].plot(kind='hist', alpha=0.5, bins=60)
plt.legend(["poi","non_poi"])
plt.ylabel("Frequency")
plt.title("salary")
plt.show()